# Enter Trip Information

In [1]:
# Enter a trip name (a unique identifier for the specified points of interest)
TRIP_NAME = "east"

# Enter your points of interest in the list below (include your starting location)
MY_POINTS_OF_INTEREST = [
    "San Francisco, California", # Starting Location
    "Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA", # Yosemite NP
    "D L Bliss State Park, California 89, South Lake Tahoe, CA", # Lake Tahoe
    "Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT", # Zion NP
    "Arches National Park Visitor Center, Moab, UT", # Arches NP
    "Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ", # Monument Valley
    "Island in the Sky Visitor Center, Grand View Point Road, Moab, UT", # Canyonlands NP
    "Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT", # Bryce Canyon NP
    "Moro Rock Trail, California", # Sequoia NP
    "Horseshoe Bend Parking Lot, Page, AZ", # Horseshoe Bend Trail
    "Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ", # Grand Canyon NP
    "Calf Creek Campground, Boulder, UT", # Grand Staircase-Escalante NM
    "Red Cliffs Recreation Area, Unnamed Road, Washington, UT", # Red Cliffs Recreation Nature Trail
    "Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT", # Natural Bridges NM
    "Kanarra Creek Trailhead, Kanarraville, UT", # Kanarra Creek Canyon Trail
    "San Simeon, CA", # South Big Sur Drive
    "Big Sur, CA", # Middle Big Sur Drive
    "Carmel-by-the-Sea, CA", # North Big Sur Drive
    "Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA", # Saratoga Gap Trail
]

# Collect Driving Distance/Duration Information

In [2]:
from src.data_collection import *
from config import GOOGLE_MAPS_API_KEY


# Determine distance and duration filename based on specified trip name
distance_duration_filename = "data/my_{}_points_of_interest_distance_duration.csv".format(TRIP_NAME)

# Try to create a distance and duration df containing all my points of interest from the filename
try:
    
    distance_duration_df = pd.read_csv(distance_duration_filename, index_col=0)

    # Create list of unique points of interest from df
    df_points_of_interest = set(pd.unique(distance_duration_df[['Venue 1', 'Venue 2']].values.ravel('K')))

    # Check if missing one or more of my points of interest in df
    if not set(MY_POINTS_OF_INTEREST).issubset(df_points_of_interest):
        
        raise Exception("Missing one or more of my points of interest in '{}'".format(distance_duration_filename))

# Create a distance and duration df with all my points of interest and save to the specified filename
except (FileNotFoundError, Exception) as e:
        
    # Query Google Maps API for one-way driving distances and durations
    distance_duration_data = query_gmaps_api_for_one_way_driving_distance_and_duration(MY_POINTS_OF_INTEREST, GOOGLE_MAPS_API_KEY)

    # Create DataFrame of one-way distances and durations
    distance_duration_df = create_distance_and_duration_df(distance_duration_data)

    # Save DataFrame to CSV
    distance_duration_df.to_csv(distance_duration_filename)
    
# Preview distance and duration df
distance_duration_df.head().sort_values('Distance (mi)', ascending=False)

,Venue 1,Venue 2,Distance (mi),Duration (s),Duration (hhmm)
3,"San Francisco, California","Arches National Park Visitor Center, Moab, UT",963,51219,14:13
4,"San Francisco, California","Monument Valley Navajo Tribal Park, Main Monum...",941,52195,14:29
2,"San Francisco, California","Zion National Park Visitor Center, Zion – Moun...",727,40019,11:06
1,"San Francisco, California","D L Bliss State Park, California 89, South Lak...",197,12542,3:29
0,"San Francisco, California","Yosemite Valley Visitor Center, Village Drive,...",191,14181,3:56


## *Optional : Display Full Name Squareform Distance/Duration Matrices*

In [3]:
# Add reverse travel information (B to A not just A to B) to distance and duration df
_df = add_reverse_travel_information_to_distance_duration_df(distance_duration_df)

# Create squareform matrices
distance_matrix = _df.pivot(index='Venue 1', columns='Venue 2', values='Distance (mi)').fillna(0).astype(int)
duration_matrix = _df.pivot(index='Venue 1', columns='Venue 2', values='Duration (s)').fillna(0).astype(int)
duration_matrix_hhmm = _df.pivot(index='Venue 1', columns='Venue 2', values='Duration (hhmm)').fillna("0:00")

In [7]:
# # Display distance matrix
# display(distance_matrix)

In [8]:
# # Display duration matrix
# display(duration_matrix)

In [6]:
# Display duration hhmm matrix
display(duration_matrix_hhmm)

Venue 2,"Arches National Park Visitor Center, Moab, UT","Big Sur, CA","Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT","Calf Creek Campground, Boulder, UT","Carmel-by-the-Sea, CA","Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA","D L Bliss State Park, California 89, South Lake Tahoe, CA","Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ","Horseshoe Bend Parking Lot, Page, AZ","Island in the Sky Visitor Center, Grand View Point Road, Moab, UT","Kanarra Creek Trailhead, Kanarraville, UT","Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ","Moro Rock Trail, California","Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT","Red Cliffs Recreation Area, Unnamed Road, Washington, UT","San Francisco, California","San Simeon, CA","Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA","Zion National Park Visitor Center, Zion – Mount Carmel Highway, Hurricane, UT"
Venue 1,,,,,,,,,,,,,,,,,,,
"Arches National Park Visitor Center, Moab, UT",0:00,15:20,4:13,3:38,14:22,14:47,12:01,5:43,4:40,0:36,4:19,2:48,13:23,2:14,4:39,14:13,13:11,12:23,5:09
"Big Sur, CA",15:20,0:00,12:48,13:54,1:03,2:20,5:56,12:20,13:12,15:37,11:10,14:38,5:37,15:57,10:51,2:57,2:21,4:57,11:28
"Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT",4:13,12:48,0:00,1:21,11:51,12:15,10:30,5:05,2:44,4:30,1:47,4:41,10:51,4:41,2:08,12:28,10:39,10:03,1:51
"Calf Creek Campground, Boulder, UT",3:38,13:54,1:21,0:00,12:57,13:21,11:36,6:07,3:50,3:54,2:53,4:50,11:56,3:40,3:14,13:34,11:45,11:09,2:57
"Carmel-by-the-Sea, CA",14:22,1:03,11:51,12:57,0:00,1:25,4:59,11:22,12:15,14:39,10:13,13:40,4:39,14:59,9:53,2:00,2:22,4:00,10:31
"Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA",14:47,2:20,12:15,13:21,1:25,0:00,4:21,11:46,12:39,15:03,10:37,14:05,4:50,15:23,10:18,1:09,3:23,4:11,10:55
"D L Bliss State Park, California 89, South Lake Tahoe, CA",12:01,5:56,10:30,11:36,4:59,4:21,0:00,12:02,11:36,12:18,9:06,13:34,6:38,13:30,9:15,3:29,6:42,4:36,9:53
"Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ",5:43,12:20,5:05,6:07,11:22,11:46,12:02,0:00,2:24,6:12,4:54,3:07,10:23,4:20,4:38,12:01,10:11,11:22,4:39
"Horseshoe Bend Parking Lot, Page, AZ",4:40,13:12,2:44,3:50,12:15,12:39,11:36,2:24,0:00,5:08,2:50,2:04,11:12,3:21,2:34,12:50,11:03,11:08,2:18


## *Optional : Display Integer Name Squareform Distance/Duration Matrices*

In [9]:
# Add reverse travel information (B to A not just A to B) to distance and duration df
_df = add_reverse_travel_information_to_distance_duration_df(distance_duration_df)

# Convert venue columns to categorical type and create categorical code columns
_df['Venue 1'] = _df['Venue 1'].astype('category')
_df['Venue 1 Codes'] = _df['Venue 1'].cat.codes
_df['Venue 2'] = pd.Categorical(_df['Venue 2'], categories=_df['Venue 1'].cat.categories)
_df['Venue 2 Codes'] = _df['Venue 2'].cat.codes

In [11]:
_df.head()

,Venue 1,Venue 2,Distance (mi),Duration (s),Duration (hhmm),Venue 1 Codes,Venue 2 Codes
0,"San Francisco, California","Yosemite Valley Visitor Center, Village Drive,...",191,14181,3:56,15,17
1,"San Francisco, California","D L Bliss State Park, California 89, South Lak...",197,12542,3:29,15,6
2,"San Francisco, California","Zion National Park Visitor Center, Zion – Moun...",727,40019,11:06,15,18
3,"San Francisco, California","Arches National Park Visitor Center, Moab, UT",963,51219,14:13,15,0
4,"San Francisco, California","Monument Valley Navajo Tribal Park, Main Monum...",941,52195,14:29,15,11


In [12]:
# Preview new columns
_df.sample(5)

,Venue 1,Venue 2,Distance (mi),Duration (s),Duration (hhmm),Venue 1 Codes,Venue 2 Codes
270,"Natural Bridges Visitor Center, Natural Bridge...","Island in the Sky Visitor Center, Grand View P...",146,9777,2:42,13,9
153,"Red Cliffs Recreation Area, Unnamed Road, Wash...","Big Sur, CA",665,39079,10:51,14,1
189,"D L Bliss State Park, California 89, South Lak...","Yosemite Valley Visitor Center, Village Drive,...",202,16572,4:36,6,17
174,"Arches National Park Visitor Center, Moab, UT","San Francisco, California",963,51219,14:13,0,15
18,"Yosemite Valley Visitor Center, Village Drive,...","D L Bliss State Park, California 89, South Lak...",202,16572,4:36,17,6


In [14]:
# Print dict of code: cat mappings for reference
_ = dict(enumerate(_df['Venue 1'].cat.categories))
for k, v in _.items():
    print(k, ":", v)

0 : Arches National Park Visitor Center, Moab, UT
1 : Big Sur, CA
2 : Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT
3 : Calf Creek Campground, Boulder, UT
4 : Carmel-by-the-Sea, CA
5 : Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA
6 : D L Bliss State Park, California 89, South Lake Tahoe, CA
7 : Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ
8 : Horseshoe Bend Parking Lot, Page, AZ
9 : Island in the Sky Visitor Center, Grand View Point Road, Moab, UT
10 : Kanarra Creek Trailhead, Kanarraville, UT
11 : Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ
12 : Moro Rock Trail, California
13 : Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT
14 : Red Cliffs Recreation Area, Unnamed Road, Washington, UT
15 : San Francisco, California
16 : San Simeon, CA
17 : Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA
18 : Zion National Park Visitor Center, Zion – Mo

In [19]:
# Create squareform matrices with codes
distance_matrix = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Distance (mi)').fillna(0).astype(int)
duration_matrix = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Duration (s)').fillna(0).astype(int)
duration_matrix_hhmm = _df.pivot(index='Venue 1 Codes', columns='Venue 2 Codes', values='Duration (hhmm)').fillna("0:00")

In [20]:
# Display distance matrix with code mappings
display(distance_matrix)

# Print dict of code: cat mappings for reference
_ = dict(enumerate(_df['Venue 1'].cat.categories))
for k, v in _.items():
    print(k, ":", v)

Venue 2 Codes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Venue 1 Codes,,,,,,,,,,,,,,,,,,,
0,0,985,246,199,961,992,808,331,280,29,296,154,849,119,322,963,882,749,341
1,985,0,793,850,26,94,323,755,807,999,690,904,266,965,665,146,80,225,692
2,246,793,0,66,768,799,607,284,154,260,103,276,657,252,130,828,689,583,85
3,199,850,66,0,826,857,665,341,212,213,161,248,715,191,187,886,747,641,143
4,961,26,768,826,0,70,298,731,782,975,666,880,241,940,641,122,91,201,668
5,992,94,799,857,70,0,246,761,813,1006,697,911,248,971,672,55,196,208,699
6,808,323,607,665,298,246,0,738,701,822,540,823,360,790,559,197,411,202,586
7,331,755,284,341,731,761,738,0,132,358,279,177,619,238,262,790,652,682,248
8,280,807,154,212,782,813,701,132,0,307,166,127,671,187,148,842,703,658,118


0 : Arches National Park Visitor Center, Moab, UT
1 : Big Sur, CA
2 : Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT
3 : Calf Creek Campground, Boulder, UT
4 : Carmel-by-the-Sea, CA
5 : Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA
6 : D L Bliss State Park, California 89, South Lake Tahoe, CA
7 : Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ
8 : Horseshoe Bend Parking Lot, Page, AZ
9 : Island in the Sky Visitor Center, Grand View Point Road, Moab, UT
10 : Kanarra Creek Trailhead, Kanarraville, UT
11 : Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ
12 : Moro Rock Trail, California
13 : Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT
14 : Red Cliffs Recreation Area, Unnamed Road, Washington, UT
15 : San Francisco, California
16 : San Simeon, CA
17 : Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA
18 : Zion National Park Visitor Center, Zion – Mo

In [21]:
# Display duration matrix with code mappings
display(duration_matrix)

# Print dict of code: cat mappings for reference
_ = dict(enumerate(_df['Venue 1'].cat.categories))
for k, v in _.items():
    print(k, ":", v)

Venue 2 Codes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Venue 1 Codes,,,,,,,,,,,,,,,,,,,
0,0,55216,15239,13081,51773,53227,43281,20596,16812,2196,15548,10084,48202,8052,16798,51219,47489,44587,18540
1,55216,0,46126,50068,3782,8457,21397,44402,47551,56222,40227,52694,20225,57420,39079,10654,8488,17847,41325
2,15239,46126,0,4876,42684,44138,37802,18314,9891,16246,6458,16910,39113,16875,7709,44894,38399,36194,6704
3,13081,50068,4876,0,46625,48079,41767,22066,13858,14087,10400,17427,42982,13242,11650,48859,42341,40159,10668
4,51773,3782,42684,46625,0,5119,17954,40959,44109,52779,36785,49252,16782,53978,35636,7211,8526,14404,37883
5,53227,8457,44138,48079,5119,0,15718,42413,45563,54233,38239,50705,17438,55432,37090,4191,12230,15060,39336
6,43281,21397,37802,41767,17954,15718,0,43372,41803,44314,32771,48851,23898,48618,33332,12542,24171,16572,35598
7,20596,44402,18314,22066,40959,42413,43372,0,8651,22321,17691,11273,37404,15650,16723,43280,36675,40953,16749
8,16812,47551,9891,13858,44109,45563,41803,8651,0,18538,10221,7490,40348,12087,9254,46224,39824,40122,8326


0 : Arches National Park Visitor Center, Moab, UT
1 : Big Sur, CA
2 : Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT
3 : Calf Creek Campground, Boulder, UT
4 : Carmel-by-the-Sea, CA
5 : Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA
6 : D L Bliss State Park, California 89, South Lake Tahoe, CA
7 : Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ
8 : Horseshoe Bend Parking Lot, Page, AZ
9 : Island in the Sky Visitor Center, Grand View Point Road, Moab, UT
10 : Kanarra Creek Trailhead, Kanarraville, UT
11 : Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ
12 : Moro Rock Trail, California
13 : Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT
14 : Red Cliffs Recreation Area, Unnamed Road, Washington, UT
15 : San Francisco, California
16 : San Simeon, CA
17 : Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA
18 : Zion National Park Visitor Center, Zion – Mo

In [22]:
# Display duration matrix hhmm with code mappings
display(duration_matrix_hhmm)

# Print dict of code: cat mappings for reference
_ = dict(enumerate(_df['Venue 1'].cat.categories))
for k, v in _.items():
    print(k, ":", v)

Venue 2 Codes,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
Venue 1 Codes,,,,,,,,,,,,,,,,,,,
0,0:00,15:20,4:13,3:38,14:22,14:47,12:01,5:43,4:40,0:36,4:19,2:48,13:23,2:14,4:39,14:13,13:11,12:23,5:09
1,15:20,0:00,12:48,13:54,1:03,2:20,5:56,12:20,13:12,15:37,11:10,14:38,5:37,15:57,10:51,2:57,2:21,4:57,11:28
2,4:13,12:48,0:00,1:21,11:51,12:15,10:30,5:05,2:44,4:30,1:47,4:41,10:51,4:41,2:08,12:28,10:39,10:03,1:51
3,3:38,13:54,1:21,0:00,12:57,13:21,11:36,6:07,3:50,3:54,2:53,4:50,11:56,3:40,3:14,13:34,11:45,11:09,2:57
4,14:22,1:03,11:51,12:57,0:00,1:25,4:59,11:22,12:15,14:39,10:13,13:40,4:39,14:59,9:53,2:00,2:22,4:00,10:31
5,14:47,2:20,12:15,13:21,1:25,0:00,4:21,11:46,12:39,15:03,10:37,14:05,4:50,15:23,10:18,1:09,3:23,4:11,10:55
6,12:01,5:56,10:30,11:36,4:59,4:21,0:00,12:02,11:36,12:18,9:06,13:34,6:38,13:30,9:15,3:29,6:42,4:36,9:53
7,5:43,12:20,5:05,6:07,11:22,11:46,12:02,0:00,2:24,6:12,4:54,3:07,10:23,4:20,4:38,12:01,10:11,11:22,4:39
8,4:40,13:12,2:44,3:50,12:15,12:39,11:36,2:24,0:00,5:08,2:50,2:04,11:12,3:21,2:34,12:50,11:03,11:08,2:18


0 : Arches National Park Visitor Center, Moab, UT
1 : Big Sur, CA
2 : Bryce Canyon National Park Visitor Center, Utah 63, Bryce, UT
3 : Calf Creek Campground, Boulder, UT
4 : Carmel-by-the-Sea, CA
5 : Castle Rock Entrance Station Parking Lot, Unnamed Road, Saratoga, CA
6 : D L Bliss State Park, California 89, South Lake Tahoe, CA
7 : Grand Canyon Visitor Center, South Entrance Road, Grand Canyon Village, AZ
8 : Horseshoe Bend Parking Lot, Page, AZ
9 : Island in the Sky Visitor Center, Grand View Point Road, Moab, UT
10 : Kanarra Creek Trailhead, Kanarraville, UT
11 : Monument Valley Navajo Tribal Park, Main Monument Valley Road, Oljato-Monument Valley, AZ
12 : Moro Rock Trail, California
13 : Natural Bridges Visitor Center, Natural Bridge, Lake Powell, UT
14 : Red Cliffs Recreation Area, Unnamed Road, Washington, UT
15 : San Francisco, California
16 : San Simeon, CA
17 : Yosemite Valley Visitor Center, Village Drive, Yosemite Valley, CA
18 : Zion National Park Visitor Center, Zion – Mo